# U.S. Medical Insurance Costs
### Analysis and Modeling by Michael dela Rosa

### GOALS:
   The goal for this study is to determine which factors most contribute to the cost of health insurance using a multivariable linear regression as well as build a model to predict a person's insurance cost based off of the available data 


In [1]:
# This is the import section
import pandas as pd 
import numpy as np
import statsmodels.api as sm 
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import scale
from scipy import stats
import statistics


## Data Cleaning and Exploration

In [2]:
# This section loads the data into a pandas data frame and inspects the header 
data = pd.DataFrame(pd.read_csv('insurance.csv'))
data.head()


,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [3]:
# Now we inspect how many observations we have 
print(data.shape)

(1338, 7)


In [4]:
# Now we are going to clean the data from NaNs if they are there, since we only want data that we can use 
clean_data = data.dropna()
print(data.shape) # Turns out there is no missing numbers so we can proceed

(1338, 7)


In [5]:
# Now we will make the data easier to use by converting categorical variables into numerical variables where applicable
Multi = data.copy() # making a copy so that I don't damage the original dataset

# since we have some categorical variables, we need to convert them into binary variables to properly use Multiple Regression
Multi = Multi.rename(columns={'sex':'Is Female'})
Multi['Is Female'].replace(['male','female'],[0,1], inplace = True)
Multi['smoker'].replace(['yes','no'],[1,0], inplace = True)

# After realizing that region cannot be properly boiled down to a numerical variable that makes sense, I decided to split the data into a per region basis
# Note: I removed the region column after separation because it is no longer needed
totalNE = Multi[Multi['region'] == 'northeast'].drop(columns = ['region'])
totalNW = Multi[Multi['region'] == 'northwest'].drop(columns = ['region'])
totalSE = Multi[Multi['region'] == 'southeast'].drop(columns = ['region'])
totalSW = Multi[Multi['region'] == 'southwest'].drop(columns = ['region'])

# Now we inspect the shape of the data so that we insure we split it correctly
print('NE shape:', totalNE.shape)
print('NW shape:', totalNW.shape)
print('SE shape:', totalSE.shape)
print('SW shape:', totalSW.shape)

total = 324+325+364+325
print('Additon for check:', total)

NE shape: (324, 6)
NW shape: (325, 6)
SE shape: (364, 6)
SW shape: (325, 6)
Additon for check: 1338


In [54]:
# Now I want to compute the summary statistics for each of the variables in each region 

# Start with the North East 

# For age, bmi, children  and charges it makes sense to look at the numerical summary statistics
print(totalNE[['age','bmi','children','charges']].describe().round(3))
print()

# For Sex and Smoker status, it makes sense to determine the total value and percentage of each respective stat
print("Total female identifying persons in this region:", totalNE['Is Female'].sum())
print("Total male identifying persons in this region:", len(totalNE['Is Female']) - totalNE['Is Female'].sum())
print("Total smokers in this region:", totalNE['smoker'].sum())
print("Total non-smokers in this region:", len(totalNE['smoker']) - totalNE['smoker'].sum())
print("Proportion of smokers to total population:",(totalNE['smoker'].sum()/len(totalNE['smoker'])).round(5))

           age      bmi  children    charges
count  324.000  324.000   324.000    324.000
mean    39.269   29.174     1.046  13406.385
std     14.069    5.938     1.199  11255.803
min     18.000   15.960     0.000   1694.796
25%     27.000   24.866     0.000   5194.322
50%     39.500   28.880     1.000  10057.652
75%     51.000   32.894     2.000  16687.364
max     64.000   48.070     5.000  58571.074

Total female identifying persons in this region: 161
Total male identifying persons in this region: 163
Total smokers in this region: 67
Total non-smokers in this region: 257
Proportion of smokers to total population: 0.20679


In [55]:
# Start with the North West

# For age, bmi, children  and charges it makes sense to look at the numerical summary statistics
print(totalNW[['age','bmi','children','charges']].describe().round(3))
print()

# For Sex and Smoker status, it makes sense to determine the total value and percentage of each respective stat
print("Total female identifying persons in this region:", totalNW['Is Female'].sum())
print("Total male identifying persons in this region:", len(totalNW['Is Female']) - totalNW['Is Female'].sum())
print("Total smokers in this region:", totalNW['smoker'].sum())
print("Total non-smokers in this region:", len(totalNW['smoker']) - totalNW['smoker'].sum())
print("Proportion of smokers to total population:",(totalNW['smoker'].sum()/len(totalNW['smoker'])).round(5))

           age      bmi  children    charges
count  325.000  325.000   325.000    325.000
mean    39.197   29.200     1.148  12417.575
std     14.052    5.137     1.172  11072.277
min     19.000   17.385     0.000   1621.340
25%     26.000   25.745     0.000   4719.737
50%     39.000   28.880     1.000   8965.796
75%     51.000   32.775     2.000  14711.744
max     64.000   42.940     5.000  60021.399

Total female identifying persons in this region: 164
Total male identifying persons in this region: 161
Total smokers in this region: 58
Total non-smokers in this region: 267
Proportion of smokers to total population: 0.17846


In [56]:
# South East

# For age, bmi, children  and charges it makes sense to look at the numerical summary statistics
print(totalSE[['age','bmi','children','charges']].describe().round(3))
print()

# For Sex and Smoker status, it makes sense to determine the total value and percentage of each respective stat
print("Total female identifying persons in this region:", totalSE['Is Female'].sum())
print("Total male identifying persons in this region:", len(totalSE['Is Female']) - totalSE['Is Female'].sum())
print("Total smokers in this region:", totalSE['smoker'].sum())
print("Total non-smokers in this region:", len(totalSE['smoker']) - totalSE['smoker'].sum())
print("Proportion of smokers to total population:",(totalSE['smoker'].sum()/len(totalSE['smoker'])).round(5))

           age      bmi  children    charges
count  364.000  364.000   364.000    364.000
mean    38.940   33.356     1.049  14735.411
std     14.165    6.478     1.177  13971.099
min     18.000   19.800     0.000   1121.874
25%     26.750   28.572     0.000   4440.886
50%     39.000   33.330     1.000   9294.132
75%     51.000   37.812     2.000  19526.287
max     64.000   53.130     5.000  63770.428

Total female identifying persons in this region: 175
Total male identifying persons in this region: 189
Total smokers in this region: 91
Total non-smokers in this region: 273
Proportion of smokers to total population: 0.25


In [57]:
# South West

# For age, bmi, children  and charges it makes sense to look at the numerical summary statistics
print(totalSW[['age','bmi','children','charges']].describe().round(3))
print()

# For Sex and Smoker status, it makes sense to determine the total value and percentage of each respective stat
print("Total female identifying persons in this region:", totalSW['Is Female'].sum())
print("Total male identifying persons in this region:", len(totalSW['Is Female']) - totalSW['Is Female'].sum())
print("Total smokers in this region:", totalSW['smoker'].sum())
print("Total non-smokers in this region:", len(totalSW['smoker']) - totalSW['smoker'].sum())
print("Proportion of smokers to total population:",(totalSW['smoker'].sum()/len(totalSW['smoker'])).round(5))

           age      bmi  children    charges
count  325.000  325.000   325.000    325.000
mean    39.455   30.597     1.142  12346.937
std     13.960    5.692     1.276  11557.179
min     19.000   17.400     0.000   1241.565
25%     27.000   26.900     0.000   4751.070
50%     39.000   30.300     1.000   8798.593
75%     51.000   34.600     2.000  13462.520
max     64.000   47.600     5.000  52590.829

Total female identifying persons in this region: 162
Total male identifying persons in this region: 163
Total smokers in this region: 58
Total non-smokers in this region: 267
Proportion of smokers to total population: 0.17846


#### Summary of Data Observation

The four different regions have relatively similar statistical makeups, with the south east having slightly higher average BMI and a slightly higher number of smokers in the region. Simply based off first impressions it seems like there is an association between bmi and charges, therefore in addition to building a model based off of what I find in the multiple regression analysis I will also build a model based solely on BMI to test my theory

## Multiple Regression

In [7]:
# Now we do a Multiple regression for each region! 
# North East 
predictors = totalNE.drop(columns = ['charges']).copy()
predictors_constant = sm.add_constant(predictors)
predictors_constant.head()
model = sm.OLS(np.array(totalNE['charges']),predictors_constant).fit()
model.summary()

# The results of the OLS lead me to beleive that gender is not a good indicator for cost of medical insurance
# This leads me to attempt to see if removing gender will result in a clearer picture


/Users/michaeldelarosa/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.709
Model:                            OLS   Adj. R-squared:                  0.705
Method:                 Least Squares   F-statistic:                     155.2
Date:                Sat, 28 Jan 2023   Prob (F-statistic):           4.52e-83
Time:                        13:29:17   Log-Likelihood:                -3281.6
No. Observations:                 324   AIC:                             6575.
Df Residuals:                     318   BIC:                             6598.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -1.346e+04   1918.548     -7.018      0.000   -1.72e+04   -9689.754
age          235.3009     24.517      9.598      0.000     187.065     283.536
Is Female    -26.1664    681.764     -0.038      0.969   -1367.505    1315.172
bmi          429.5411     58.140      7.388      0.000     315.153     543.929
children     707.9024    284.079      2.492      0.013     148.990    1266.815
smoker      2.114e+04    842.254     25.102      0.000    1.95e+04    2.28e+04
==============================================================================
Omnibus:                       78.874   Durbin-Watson:                   2.003
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              150.750
Skew:                           1.297   Prob(JB):                     1.84e-33
Kurtosis:                       5.107   Cond. No.                         287.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [8]:
# North East 
predictors = totalNE.drop(columns = ['charges', 'Is Female']).copy()
predictors_constant = sm.add_constant(predictors)
predictors_constant.head()
model = sm.OLS(np.array(totalNE['charges']),predictors_constant).fit()
model.summary()
# Once again, most of the other variables had significant p-values for their constants, suggesting that they have influence on the cost of medical insurance  

/Users/michaeldelarosa/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.709
Model:                            OLS   Adj. R-squared:                  0.706
Method:                 Least Squares   F-statistic:                     194.6
Date:                Sat, 28 Jan 2023   Prob (F-statistic):           3.04e-84
Time:                        13:29:17   Log-Likelihood:                -3281.6
No. Observations:                 324   AIC:                             6573.
Df Residuals:                     319   BIC:                             6592.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -1.348e+04   1890.417     -7.129      0.000   -1.72e+04   -9757.032
age          235.2816     24.473      9.614      0.000     187.132     283.431
bmi          429.5015     58.040      7.400      0.000     315.312     543.691
children     708.2607    283.481      2.498      0.013     150.532    1265.990
smoker      2.114e+04    839.227     25.195      0.000    1.95e+04    2.28e+04
==============================================================================
Omnibus:                       78.808   Durbin-Watson:                   2.003
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              150.530
Skew:                           1.296   Prob(JB):                     2.05e-33
Kurtosis:                       5.105   Cond. No.                         282.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [9]:
# North West 

predictors = totalNW.drop(columns = ['charges']).copy()
predictors_constant = sm.add_constant(predictors)
predictors_constant.head()
model = sm.OLS(np.array(totalNW['charges']),predictors_constant).fit()
model.summary()

# Its the same story in the north west as well

/Users/michaeldelarosa/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.704
Model:                            OLS   Adj. R-squared:                  0.700
Method:                 Least Squares   F-statistic:                     152.0
Date:                Sat, 28 Jan 2023   Prob (F-statistic):           3.50e-82
Time:                        13:29:17   Log-Likelihood:                -3289.1
No. Observations:                 325   AIC:                             6590.
Df Residuals:                     319   BIC:                             6613.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -1.124e+04   2114.551     -5.313      0.000   -1.54e+04   -7075.075
age          247.1956     24.163     10.230      0.000     199.656     294.735
Is Female      5.0752    673.764      0.008      0.994   -1320.507    1330.658
bmi          313.2682     66.175      4.734      0.000     183.073     443.463
children     853.2469    289.605      2.946      0.003     283.469    1423.025
smoker      2.149e+04    880.139     24.412      0.000    1.98e+04    2.32e+04
==============================================================================
Omnibus:                      117.813   Durbin-Watson:                   2.117
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              314.569
Skew:                           1.732   Prob(JB):                     4.92e-69
Kurtosis:                       6.351   Cond. No.                         317.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [10]:
# South East 
predictors = totalSE.drop(columns = ['charges']).copy()
predictors_constant = sm.add_constant(predictors)
predictors_constant.head()
model = sm.OLS(np.array(totalSE['charges']),predictors_constant).fit()
model.summary()

/Users/michaeldelarosa/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.799
Model:                            OLS   Adj. R-squared:                  0.796
Method:                 Least Squares   F-statistic:                     284.8
Date:                Sat, 28 Jan 2023   Prob (F-statistic):          2.20e-122
Time:                        13:29:17   Log-Likelihood:                -3698.2
No. Observations:                 364   AIC:                             7408.
Df Residuals:                     358   BIC:                             7432.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       -1.46e+04   2036.253     -7.168      0.000   -1.86e+04   -1.06e+04
age          271.9832     23.550     11.549      0.000     225.669     318.298
Is Female    259.6546    668.485      0.388      0.698   -1054.997    1574.306
bmi          343.0524     51.437      6.669      0.000     241.896     444.209
children     473.5471    283.361      1.671      0.096     -83.715    1030.809
smoker       2.67e+04    768.193     34.761      0.000    2.52e+04    2.82e+04
==============================================================================
Omnibus:                       62.817   Durbin-Watson:                   2.111
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              147.150
Skew:                           0.863   Prob(JB):                     1.11e-32
Kurtosis:                       5.593   Cond. No.                         326.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [11]:
# South West 
predictors = totalSW.drop(columns = ['charges']).copy()
predictors_constant = sm.add_constant(predictors)
predictors_constant.head()
model = sm.OLS(np.array(totalSW['charges']),predictors_constant).fit()
model.summary()

/Users/michaeldelarosa/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.786
Model:                            OLS   Adj. R-squared:                  0.783
Method:                 Least Squares   F-statistic:                     234.3
Date:                Sat, 28 Jan 2023   Prob (F-statistic):          1.74e-104
Time:                        13:29:17   Log-Likelihood:                -3250.5
No. Observations:                 325   AIC:                             6513.
Df Residuals:                     319   BIC:                             6536.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -1.097e+04   1789.725     -6.129      0.000   -1.45e+04   -7447.211
age          269.0785     22.121     12.164      0.000     225.557     312.600
Is Female    468.7823    605.363      0.774      0.439    -722.226    1659.790
bmi          258.9068     54.277      4.770      0.000     152.121     365.693
children      25.6151    235.237      0.109      0.913    -437.196     488.427
smoker      2.529e+04    792.889     31.902      0.000    2.37e+04    2.69e+04
==============================================================================
Omnibus:                       84.587   Durbin-Watson:                   1.890
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              278.780
Skew:                           1.127   Prob(JB):                     2.91e-61
Kurtosis:                       6.938   Cond. No.                         310.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Curiously, in the south west, it seems like children also is not a large contributing factor to the cost of insurance for individuals! Let's see what happens if we remove children and Gender.

In [12]:
# North East 
predictors = totalNE.drop(columns = ['charges', 'Is Female', 'children']).copy()
predictors_constant = sm.add_constant(predictors)
predictors_constant.head()
model = sm.OLS(np.array(totalNE['charges']),predictors_constant).fit()
model.summary()

/Users/michaeldelarosa/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.704
Model:                            OLS   Adj. R-squared:                  0.701
Method:                 Least Squares   F-statistic:                     253.2
Date:                Sat, 28 Jan 2023   Prob (F-statistic):           3.83e-84
Time:                        13:29:17   Log-Likelihood:                -3284.7
No. Observations:                 324   AIC:                             6577.
Df Residuals:                     320   BIC:                             6593.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -1.264e+04   1875.589     -6.738      0.000   -1.63e+04   -8948.083
age          235.0203     24.673      9.526      0.000     186.479     283.561
bmi          426.5523     58.501      7.291      0.000     311.456     541.648
smoker      2.114e+04    846.072     24.986      0.000    1.95e+04    2.28e+04
==============================================================================
Omnibus:                       80.593   Durbin-Watson:                   2.023
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              157.335
Skew:                           1.312   Prob(JB):                     6.84e-35
Kurtosis:                       5.184   Cond. No.                         278.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

All of the other variables seem to predict the cost relatively well! Now all we have to do is build that prediction model and test its accuracy using kfold 

## PREDICTION!

Through our observations above, we found that all predictors except gender most likely estimate cost per person for reigons NE,NW,and SE. For SW we found that the predictors besides gender and children most likely estimate cost per person for that region. For each region I will be building one prediction model with all predictors, and one without the ones I determined were not useful and testing each for average R^2 and RMSE to compare performance.

In [13]:
# North East with all

predictors = totalNE.drop(columns = ['charges']).copy()

# First we need to prepare the data for cross valudation 
kfold = KFold(10) # using the standard 10 groups 
total_RMSElasso = []
total_rsquaredlasso = []
total_RMSELinear = []
total_rsquaredLinear = []


for train, test in kfold.split(predictors):
    trainingX = pd.DataFrame()
    trainingY = []
    testingX = pd.DataFrame()
    testingY = []
    for ii in train:
        trainingX = pd.concat([trainingX,pd.DataFrame(predictors.iloc[ii]).transpose()])
        trainingY.append(totalNE['charges'].iloc[ii])
    for jj in test:
        testingX = pd.concat([testingX,pd.DataFrame(predictors.iloc[jj]).transpose()])
        testingY.append(totalNE['charges'].iloc[jj])
    
    #Performing a Lasso Regression and a Linear Regression and comparing the two
    lasso = Lasso(max_iter = 10000, normalize = True)
    lasso.set_params(alpha = 0.01)
    lasso.fit(pd.DataFrame(trainingX), trainingY)
    predictionLass = lasso.predict(pd.DataFrame(testingX))
    total_RMSElasso.append(mean_squared_error(testingY,predictionLass,squared = False ))
    total_rsquaredlasso.append(r2_score(testingY,predictionLass))
    
    linReg = LinearRegression().fit(pd.DataFrame(trainingX),y = trainingY)
    predictionLin = linReg.predict(pd.DataFrame(testingX))
    total_RMSELinear.append(mean_squared_error(testingY,predictionLin,squared = False ))
    total_rsquaredLinear.append(r2_score(testingY,predictionLin))

print('Mean RMSE for Lasso Regression:', statistics.mean(total_RMSElasso))
print('Mean  r^2 for Lasso Regression:', statistics.mean(total_rsquaredlasso))
print('Mean RMSE for Linear Regression:', statistics.mean(total_RMSELinear))
print('Mean  r^2 for Linear Regression:', statistics.mean(total_rsquaredLinear))

Mean RMSE for Lasso Regression: 6128.112585396616
Mean  r^2 for Lasso Regression: 0.6742367350902642
Mean RMSE for Linear Regression: 6128.1423087381945
Mean  r^2 for Linear Regression: 0.6742325641650744


In [14]:
# North East without gender

predictors = totalNE.drop(columns = ['charges', 'Is Female']).copy()

# First we need to prepare the data for cross valudation 
kfold = KFold(10) # using the standard 10 groups 
total_RMSElasso = []
total_rsquaredlasso = []
total_RMSELinear = []
total_rsquaredLinear = []


for train, test in kfold.split(predictors):
    trainingX = pd.DataFrame()
    trainingY = []
    testingX = pd.DataFrame()
    testingY = []
    for ii in train:
        trainingX = pd.concat([trainingX,pd.DataFrame(predictors.iloc[ii]).transpose()])
        trainingY.append(totalNE['charges'].iloc[ii])
    for jj in test:
        testingX = pd.concat([testingX,pd.DataFrame(predictors.iloc[jj]).transpose()])
        testingY.append(totalNE['charges'].iloc[jj])
    
    #Performing a Lasso Regression and a Linear Regression and comparing the two
    lasso = Lasso(max_iter = 10000, normalize = True)
    lasso.set_params(alpha = 0.01)
    lasso.fit(pd.DataFrame(trainingX), trainingY)
    predictionLass = lasso.predict(pd.DataFrame(testingX))
    total_RMSElasso.append(mean_squared_error(testingY,predictionLass,squared = False ))
    total_rsquaredlasso.append(r2_score(testingY,predictionLass))
    
    linReg = LinearRegression().fit(pd.DataFrame(trainingX),y = trainingY)
    predictionLin = linReg.predict(pd.DataFrame(testingX))
    total_RMSELinear.append(mean_squared_error(testingY,predictionLin,squared = False ))
    total_rsquaredLinear.append(r2_score(testingY,predictionLin))

print('Mean RMSE for Lasso Regression:', statistics.mean(total_RMSElasso))
print('Mean  r^2 for Lasso Regression:', statistics.mean(total_rsquaredlasso))
print('Mean RMSE for Linear Regression:', statistics.mean(total_RMSELinear))
print('Mean  r^2 for Linear Regression:', statistics.mean(total_rsquaredLinear))

Mean RMSE for Lasso Regression: 6105.511400016215
Mean  r^2 for Lasso Regression: 0.6768045874970946
Mean RMSE for Linear Regression: 6105.514596689371
Mean  r^2 for Linear Regression: 0.6768033212902821


In [15]:
# North West with all

predictors = totalNW.drop(columns = ['charges']).copy()

# First we need to prepare the data for cross valudation 
kfold = KFold(10) # using the standard 10 groups 
total_RMSElasso = []
total_rsquaredlasso = []
total_RMSELinear = []
total_rsquaredLinear = []


for train, test in kfold.split(predictors):
    trainingX = pd.DataFrame()
    trainingY = []
    testingX = pd.DataFrame()
    testingY = []
    for ii in train:
        trainingX = pd.concat([trainingX,pd.DataFrame(predictors.iloc[ii]).transpose()])
        trainingY.append(totalNW['charges'].iloc[ii])
    for jj in test:
        testingX = pd.concat([testingX,pd.DataFrame(predictors.iloc[jj]).transpose()])
        testingY.append(totalNW['charges'].iloc[jj])
    
    #Performing a Lasso Regression and a Linear Regression and comparing the two
    lasso = Lasso(max_iter = 10000, normalize = True)
    lasso.set_params(alpha = 0.01)
    lasso.fit(pd.DataFrame(trainingX), trainingY)
    predictionLass = lasso.predict(pd.DataFrame(testingX))
    total_RMSElasso.append(mean_squared_error(testingY,predictionLass,squared = False ))
    total_rsquaredlasso.append(r2_score(testingY,predictionLass))
    
    linReg = LinearRegression().fit(pd.DataFrame(trainingX),y = trainingY)
    predictionLin = linReg.predict(pd.DataFrame(testingX))
    total_RMSELinear.append(mean_squared_error(testingY,predictionLin,squared = False ))
    total_rsquaredLinear.append(r2_score(testingY,predictionLin))

print('Mean RMSE for Lasso Regression:', statistics.mean(total_RMSElasso))
print('Mean  r^2 for Lasso Regression:', statistics.mean(total_rsquaredlasso))
print('Mean RMSE for Linear Regression:', statistics.mean(total_RMSELinear))
print('Mean  r^2 for Linear Regression:', statistics.mean(total_rsquaredLinear))

Mean RMSE for Lasso Regression: 6171.414725675082
Mean  r^2 for Lasso Regression: 0.6418221705882858
Mean RMSE for Linear Regression: 6171.439644661241
Mean  r^2 for Linear Regression: 0.6418166504348506


In [16]:
# North East without gender

predictors = totalNW.drop(columns = ['charges', 'Is Female']).copy()

# First we need to prepare the data for cross valudation 
kfold = KFold(10) # using the standard 10 groups 
total_RMSElasso = []
total_rsquaredlasso = []
total_RMSELinear = []
total_rsquaredLinear = []


for train, test in kfold.split(predictors):
    trainingX = pd.DataFrame()
    trainingY = []
    testingX = pd.DataFrame()
    testingY = []
    for ii in train:
        trainingX = pd.concat([trainingX,pd.DataFrame(predictors.iloc[ii]).transpose()])
        trainingY.append(totalNW['charges'].iloc[ii])
    for jj in test:
        testingX = pd.concat([testingX,pd.DataFrame(predictors.iloc[jj]).transpose()])
        testingY.append(totalNW['charges'].iloc[jj])
    
    #Performing a Lasso Regression and a Linear Regression and comparing the two
    lasso = Lasso(max_iter = 10000, normalize = True)
    lasso.set_params(alpha = 1)
    lasso.fit(pd.DataFrame(trainingX), trainingY)
    predictionLass = lasso.predict(pd.DataFrame(testingX))
    total_RMSElasso.append(mean_squared_error(testingY,predictionLass,squared = False ))
    total_rsquaredlasso.append(r2_score(testingY,predictionLass))
    
    linReg = LinearRegression().fit(pd.DataFrame(trainingX),y = trainingY)
    predictionLin = linReg.predict(pd.DataFrame(testingX))
    total_RMSELinear.append(mean_squared_error(testingY,predictionLin,squared = False ))
    total_rsquaredLinear.append(r2_score(testingY,predictionLin))

print('Mean RMSE for Lasso Regression:', statistics.mean(total_RMSElasso))
print('Mean  r^2 for Lasso Regression:', statistics.mean(total_rsquaredlasso))
print('Mean RMSE for Linear Regression:', statistics.mean(total_RMSELinear))
print('Mean  r^2 for Linear Regression:', statistics.mean(total_rsquaredLinear))

Mean RMSE for Lasso Regression: 6142.242707653372
Mean  r^2 for Lasso Regression: 0.6451189885567237
Mean RMSE for Linear Regression: 6141.493945509427
Mean  r^2 for Linear Regression: 0.6448942181650668


In [17]:
# South East with all

predictors = totalSE.drop(columns = ['charges']).copy()

# First we need to prepare the data for cross valudation 
kfold = KFold(10) # using the standard 10 groups 
total_RMSElasso = []
total_rsquaredlasso = []
total_RMSELinear = []
total_rsquaredLinear = []


for train, test in kfold.split(predictors):
    trainingX = pd.DataFrame()
    trainingY = []
    testingX = pd.DataFrame()
    testingY = []
    for ii in train:
        trainingX = pd.concat([trainingX,pd.DataFrame(predictors.iloc[ii]).transpose()])
        trainingY.append(totalSE['charges'].iloc[ii])
    for jj in test:
        testingX = pd.concat([testingX,pd.DataFrame(predictors.iloc[jj]).transpose()])
        testingY.append(totalSE['charges'].iloc[jj])
    
    #Performing a Lasso Regression and a Linear Regression and comparing the two
    lasso = Lasso(max_iter = 10000, normalize = True)
    lasso.set_params(alpha = 0.1)
    lasso.fit(pd.DataFrame(trainingX), trainingY)
    predictionLass = lasso.predict(pd.DataFrame(testingX))
    total_RMSElasso.append(mean_squared_error(testingY,predictionLass,squared = False ))
    total_rsquaredlasso.append(r2_score(testingY,predictionLass))
    
    linReg = LinearRegression().fit(pd.DataFrame(trainingX),y = trainingY)
    predictionLin = linReg.predict(pd.DataFrame(testingX))
    total_RMSELinear.append(mean_squared_error(testingY,predictionLin,squared = False ))
    total_rsquaredLinear.append(r2_score(testingY,predictionLin))

print('Mean RMSE for Lasso Regression:', statistics.mean(total_RMSElasso))
print('Mean  r^2 for Lasso Regression:', statistics.mean(total_rsquaredlasso))
print('Mean RMSE for Linear Regression:', statistics.mean(total_RMSELinear))
print('Mean  r^2 for Linear Regression:', statistics.mean(total_rsquaredLinear))

Mean RMSE for Lasso Regression: 6326.629256133633
Mean  r^2 for Lasso Regression: 0.780044852661116
Mean RMSE for Linear Regression: 6326.774822993651
Mean  r^2 for Linear Regression: 0.7800260516212532


In [18]:
# South East without Gender

predictors = totalSE.drop(columns = ['charges', 'Is Female']).copy()

# First we need to prepare the data for cross valudation 
kfold = KFold(10) # using the standard 10 groups 
total_RMSElasso = []
total_rsquaredlasso = []
total_RMSELinear = []
total_rsquaredLinear = []


for train, test in kfold.split(predictors):
    trainingX = pd.DataFrame()
    trainingY = []
    testingX = pd.DataFrame()
    testingY = []
    for ii in train:
        trainingX = pd.concat([trainingX,pd.DataFrame(predictors.iloc[ii]).transpose()])
        trainingY.append(totalSE['charges'].iloc[ii])
    for jj in test:
        testingX = pd.concat([testingX,pd.DataFrame(predictors.iloc[jj]).transpose()])
        testingY.append(totalSE['charges'].iloc[jj])
    
    #Performing a Lasso Regression and a Linear Regression and comparing the two
    lasso = Lasso(max_iter = 10000, normalize = True)
    lasso.set_params(alpha = 0.01)
    lasso.fit(pd.DataFrame(trainingX), trainingY)
    predictionLass = lasso.predict(pd.DataFrame(testingX))
    total_RMSElasso.append(mean_squared_error(testingY,predictionLass,squared = False ))
    total_rsquaredlasso.append(r2_score(testingY,predictionLass))
    
    linReg = LinearRegression().fit(pd.DataFrame(trainingX),y = trainingY)
    predictionLin = linReg.predict(pd.DataFrame(testingX))
    total_RMSELinear.append(mean_squared_error(testingY,predictionLin,squared = False ))
    total_rsquaredLinear.append(r2_score(testingY,predictionLin))

print('Mean RMSE for Lasso Regression:', statistics.mean(total_RMSElasso))
print('Mean  r^2 for Lasso Regression:', statistics.mean(total_rsquaredlasso))
print('Mean RMSE for Linear Regression:', statistics.mean(total_RMSELinear))
print('Mean  r^2 for Linear Regression:', statistics.mean(total_rsquaredLinear))

Mean RMSE for Lasso Regression: 6312.722522364933
Mean  r^2 for Lasso Regression: 0.7810717632614991
Mean RMSE for Linear Regression: 6312.720072404691
Mean  r^2 for Linear Regression: 0.7810711858886251


In [19]:
# South West with all

predictors = totalSW.drop(columns = ['charges']).copy()

# First we need to prepare the data for cross valudation 
kfold = KFold(10) # using the standard 10 groups 
total_RMSElasso = []
total_rsquaredlasso = []
total_RMSELinear = []
total_rsquaredLinear = []


for train, test in kfold.split(predictors):
    trainingX = pd.DataFrame()
    trainingY = []
    testingX = pd.DataFrame()
    testingY = []
    for ii in train:
        trainingX = pd.concat([trainingX,pd.DataFrame(predictors.iloc[ii]).transpose()])
        trainingY.append(totalSW['charges'].iloc[ii])
    for jj in test:
        testingX = pd.concat([testingX,pd.DataFrame(predictors.iloc[jj]).transpose()])
        testingY.append(totalSW['charges'].iloc[jj])
    
    #Performing a Lasso Regression and a Linear Regression and comparing the two
    lasso = Lasso(max_iter = 10000, normalize = True)
    lasso.set_params(alpha = 0.01)
    lasso.fit(pd.DataFrame(trainingX), trainingY)
    predictionLass = lasso.predict(pd.DataFrame(testingX))
    total_RMSElasso.append(mean_squared_error(testingY,predictionLass,squared = False ))
    total_rsquaredlasso.append(r2_score(testingY,predictionLass))
    
    linReg = LinearRegression().fit(pd.DataFrame(trainingX),y = trainingY)
    predictionLin = linReg.predict(pd.DataFrame(testingX))
    total_RMSELinear.append(mean_squared_error(testingY,predictionLin,squared = False ))
    total_rsquaredLinear.append(r2_score(testingY,predictionLin))

print('Mean RMSE for Lasso Regression:', statistics.mean(total_RMSElasso))
print('Mean  r^2 for Lasso Regression:', statistics.mean(total_rsquaredlasso))
print('Mean RMSE for Linear Regression:', statistics.mean(total_RMSELinear))
print('Mean  r^2 for Linear Regression:', statistics.mean(total_rsquaredLinear))

Mean RMSE for Lasso Regression: 5353.047924670744
Mean  r^2 for Lasso Regression: 0.754335747658582
Mean RMSE for Linear Regression: 5353.071955449213
Mean  r^2 for Linear Regression: 0.7543319327692101


In [20]:
# South West with out Gender 

predictors = totalSW.drop(columns = ['charges', 'Is Female']).copy()

# First we need to prepare the data for cross valudation 
kfold = KFold(10) # using the standard 10 groups 
total_RMSElasso = []
total_rsquaredlasso = []
total_RMSELinear = []
total_rsquaredLinear = []


for train, test in kfold.split(predictors):
    trainingX = pd.DataFrame()
    trainingY = []
    testingX = pd.DataFrame()
    testingY = []
    for ii in train:
        trainingX = pd.concat([trainingX,pd.DataFrame(predictors.iloc[ii]).transpose()])
        trainingY.append(totalSW['charges'].iloc[ii])
    for jj in test:
        testingX = pd.concat([testingX,pd.DataFrame(predictors.iloc[jj]).transpose()])
        testingY.append(totalSW['charges'].iloc[jj])
    
    #Performing a Lasso Regression and a Linear Regression and comparing the two
    lasso = Lasso(max_iter = 10000, normalize = True)
    lasso.set_params(alpha = 0.01)
    lasso.fit(pd.DataFrame(trainingX), trainingY)
    predictionLass = lasso.predict(pd.DataFrame(testingX))
    total_RMSElasso.append(mean_squared_error(testingY,predictionLass,squared = False ))
    total_rsquaredlasso.append(r2_score(testingY,predictionLass))
    
    linReg = LinearRegression().fit(pd.DataFrame(trainingX),y = trainingY)
    predictionLin = linReg.predict(pd.DataFrame(testingX))
    total_RMSELinear.append(mean_squared_error(testingY,predictionLin,squared = False ))
    total_rsquaredLinear.append(r2_score(testingY,predictionLin))

print('Mean RMSE for Lasso Regression:', statistics.mean(total_RMSElasso))
print('Mean  r^2 for Lasso Regression:', statistics.mean(total_rsquaredlasso))
print('Mean RMSE for Linear Regression:', statistics.mean(total_RMSELinear))
print('Mean  r^2 for Linear Regression:', statistics.mean(total_rsquaredLinear))

Mean RMSE for Lasso Regression: 5352.5492997991105
Mean  r^2 for Lasso Regression: 0.7544961462465056
Mean RMSE for Linear Regression: 5352.577980648309
Mean  r^2 for Linear Regression: 0.7544919407470405


In [21]:
# South West without children

predictors = totalSW.drop(columns = ['charges', 'children']).copy()

# First we need to prepare the data for cross valudation 
kfold = KFold(10) # using the standard 10 groups 
total_RMSElasso = []
total_rsquaredlasso = []
total_RMSELinear = []
total_rsquaredLinear = []


for train, test in kfold.split(predictors):
    trainingX = pd.DataFrame()
    trainingY = []
    testingX = pd.DataFrame()
    testingY = []
    for ii in train:
        trainingX = pd.concat([trainingX,pd.DataFrame(predictors.iloc[ii]).transpose()])
        trainingY.append(totalSW['charges'].iloc[ii])
    for jj in test:
        testingX = pd.concat([testingX,pd.DataFrame(predictors.iloc[jj]).transpose()])
        testingY.append(totalSW['charges'].iloc[jj])
    
    #Performing a Lasso Regression and a Linear Regression and comparing the two
    lasso = Lasso(max_iter = 10000, normalize = True)
    lasso.set_params(alpha = 0.01)
    lasso.fit(pd.DataFrame(trainingX), trainingY)
    predictionLass = lasso.predict(pd.DataFrame(testingX))
    total_RMSElasso.append(mean_squared_error(testingY,predictionLass,squared = False ))
    total_rsquaredlasso.append(r2_score(testingY,predictionLass))
    
    linReg = LinearRegression().fit(pd.DataFrame(trainingX),y = trainingY)
    predictionLin = linReg.predict(pd.DataFrame(testingX))
    total_RMSELinear.append(mean_squared_error(testingY,predictionLin,squared = False ))
    total_rsquaredLinear.append(r2_score(testingY,predictionLin))

print('Mean RMSE for Lasso Regression:', statistics.mean(total_RMSElasso))
print('Mean  r^2 for Lasso Regression:', statistics.mean(total_rsquaredlasso))
print('Mean RMSE for Linear Regression:', statistics.mean(total_RMSELinear))
print('Mean  r^2 for Linear Regression:', statistics.mean(total_rsquaredLinear))

Mean RMSE for Lasso Regression: 5335.8289037687255
Mean  r^2 for Lasso Regression: 0.7560029345874892
Mean RMSE for Linear Regression: 5335.826609332393
Mean  r^2 for Linear Regression: 0.7560016680400565


In [22]:
# South West without Gender or Children

predictors = totalSW.drop(columns = ['charges', 'Is Female','children']).copy()

# First we need to prepare the data for cross valudation 
kfold = KFold(10) # using the standard 10 groups 
total_RMSElasso = []
total_rsquaredlasso = []
total_RMSELinear = []
total_rsquaredLinear = []


for train, test in kfold.split(predictors):
    trainingX = pd.DataFrame()
    trainingY = []
    testingX = pd.DataFrame()
    testingY = []
    for ii in train:
        trainingX = pd.concat([trainingX,pd.DataFrame(predictors.iloc[ii]).transpose()])
        trainingY.append(totalSW['charges'].iloc[ii])
    for jj in test:
        testingX = pd.concat([testingX,pd.DataFrame(predictors.iloc[jj]).transpose()])
        testingY.append(totalSW['charges'].iloc[jj])
    
    #Performing a Lasso Regression and a Linear Regression and comparing the two
    lasso = Lasso(max_iter = 10000, normalize = True)
    lasso.set_params(alpha = 0.01)
    lasso.fit(pd.DataFrame(trainingX), trainingY)
    predictionLass = lasso.predict(pd.DataFrame(testingX))
    total_RMSElasso.append(mean_squared_error(testingY,predictionLass,squared = False ))
    total_rsquaredlasso.append(r2_score(testingY,predictionLass))
    
    linReg = LinearRegression().fit(pd.DataFrame(trainingX),y = trainingY)
    predictionLin = linReg.predict(pd.DataFrame(testingX))
    total_RMSELinear.append(mean_squared_error(testingY,predictionLin,squared = False ))
    total_rsquaredLinear.append(r2_score(testingY,predictionLin))

print('Mean RMSE for Lasso Regression:', statistics.mean(total_RMSElasso))
print('Mean  r^2 for Lasso Regression:', statistics.mean(total_rsquaredlasso))
print('Mean RMSE for Linear Regression:', statistics.mean(total_RMSELinear))
print('Mean  r^2 for Linear Regression:', statistics.mean(total_rsquaredLinear))

Mean RMSE for Lasso Regression: 5335.0776966493395
Mean  r^2 for Lasso Regression: 0.7561826557925152
Mean RMSE for Linear Regression: 5335.0796680403055
Mean  r^2 for Linear Regression: 0.7561810452132336


## BONUS: JUST BMI

In [60]:
# Now I will examine my theory about BMI 

predictors = totalNE['bmi'].copy()

# First we need to prepare the data for cross valudation 
kfold = KFold(10) # using the standard 10 groups 
total_RMSElasso = []
total_rsquaredlasso = []
total_RMSELinear = []
total_rsquaredLinear = []


for train, test in kfold.split(predictors):
    trainingX = []
    trainingY = []
    testingX = []
    testingY = []
    for ii in train:
        trainingX.append(predictors.iloc[ii])
        trainingY.append(totalNE['charges'].iloc[ii])
    for jj in test:
        testingX.append(predictors.iloc[jj])
        testingY.append(totalNE['charges'].iloc[jj])
    
    #Performing a Lasso Regression and a Linear Regression and comparing the two
    lasso = Lasso(max_iter = 10000, normalize = True)
    lasso.set_params(alpha = 0.01)
    lasso.fit(pd.DataFrame(trainingX), trainingY)
    predictionLass = lasso.predict(pd.DataFrame(testingX))
    total_RMSElasso.append(mean_squared_error(testingY,predictionLass,squared = False ))
    total_rsquaredlasso.append(r2_score(testingY,predictionLass))
    
    linReg = LinearRegression().fit(pd.DataFrame(trainingX),y = trainingY)
    predictionLin = linReg.predict(pd.DataFrame(testingX))
    total_RMSELinear.append(mean_squared_error(testingY,predictionLin,squared = False ))
    total_rsquaredLinear.append(r2_score(testingY,predictionLin))

print('Mean RMSE for Lasso Regression:', statistics.mean(total_RMSElasso))
print('Mean  r^2 for Lasso Regression:', statistics.mean(total_rsquaredlasso))
print('Mean RMSE for Linear Regression:', statistics.mean(total_RMSELinear))
print('Mean  r^2 for Linear Regression:', statistics.mean(total_rsquaredLinear))

Mean RMSE for Lasso Regression: 10935.772532146912
Mean  r^2 for Lasso Regression: 0.010885314836867754
Mean RMSE for Linear Regression: 10935.773299550949
Mean  r^2 for Linear Regression: 0.01088478034834448


### WARNING
Based off hte first model built alone, it seems like building a model based off of just BMI is much worse than with other variables included.

### CONCLUSION

All models performed relatively well in predicting the cost of healthcare, with the best performing models coming from the two southern regions and the worse performing models coming from the northern regions (NE r^2 = .674, NW r^2 = 0.642, SE r^2 = 0.780, SW r^2 = 0.754). Removing the variables that we discovered were not very predictive of the cost from the analysis above had relatively little effect on the performance of the models, with a .003 swing in r squared being the highest difference in r^2 between models that included gender and those that did not (North West 0.642 -> 0.645). That is to be expected as removing a predictor that has no effect on the outcome would result in little change to model performance. 

Therefore, and relatively effective model at predicting health insurance costs of a person can be built from just bmi, age, children, and smoking status for all regions except the south west. For the south west region, an effective model can be constructed simply with bmi, age, and smoking status.

From this study, further questions can be asked like why do the models in the souther region perform better with the predictors given to us than the norther region. What I'm most interested in is why the number of children of a person in south west region in particular has little bearing on their insurance cost. This can be explored with more data on the region itself. 